# Reading JSON input file and extracting data

In [172]:
import json   
import pandas as pd

In [173]:
f1 = open("Sample_Data.json" , 'r')

lines = f1.readlines()

tweets = {}
tweets['tweets'] = []

for line in lines:
    if line[0] == '{':
        tweets['tweets'].append(json.loads(line)) 

In [174]:
tweet = tweets['tweets']
#tweet[0]
#tweet[0]['retweeted_status']['user']

In [175]:
#df_twitter = pd.DataFrame(tweet)

In [176]:
#df_twitter.head(2)

In [177]:
#df_twitter_user = df_twitter[['user']]
#df_twitter_user.head(5)

In [178]:
tweet[0]['user']['id_str']

'1242817830946508801'

In [179]:
len(tweet)

2001

# Connection to postgresql database

In [180]:
import psycopg2 # psycopg2 is a PostgreSQL database adapter for the Python , and it provides a Python interface for connecting to and interacting with PostgreSQL databases.

In [181]:

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="TwitterDatabase",
    user="postgres",
    password="India@2194"
)


### Creating a cursor object to execute PostgreSQL commands

In [182]:
# Create a cursor object to execute PostgreSQL commands
cursor = conn.cursor()

#### Creating a list(cache) to check and avoid adding duplicate records into the database

In [183]:
tweet_user_list = list()

# Inserting input data into the relational database

In [184]:
print("Total rows of data available: ",len(tweet))

inserted_rows_count = 0
Skipped_rows_count = 0

user_count = len(tweet)

for i in range(0,len(tweet)):
    try:
        if (tweet[i]['user']['id_str']) not in tweet_user_list:
            
            
            if  user_count > 0 :
                
            # Inserting user data user field
                cursor.execute("INSERT INTO users (id_str,screen_name,description,verified,followers_count,friends_count ,listed_count ,favourites_count,statuses_count,created_at ,protected) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (tweet[i]['user']['id_str'],tweet[i]['user']['screen_name'],tweet[i]['user']['description'],tweet[i]['user']['verified'],tweet[i]['user']['followers_count'],tweet[i]['user']['friends_count'],tweet[i]['user']['listed_count'],tweet[i]['user']['favourites_count'],tweet[i]['user']['statuses_count'],tweet[i]['user']['created_at'],tweet[i]['user']['protected'] ))
                #print('Record inserted --------------------------------------------------------------------------------->')
                tweet_user_list.append(tweet[i]['user']['id_str'])
                inserted_rows_count = inserted_rows_count + 1
            
            # Inserting user data from Retweeted status field
            else:                
                cursor.execute("INSERT INTO users (id_str,screen_name,description,verified,followers_count,friends_count ,listed_count ,favourites_count,statuses_count,created_at ,protected) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (tweet[i]['retweeted_status']['user']['id_str'],tweet[i]['retweeted_status']['user']['screen_name'],tweet[i]['retweeted_status']['user']['description'],tweet[i]['user']['verified'],tweet[i]['retweeted_status']['user']['followers_count'],tweet[i]['retweeted_status']['user']['friends_count'],tweet[i]['retweeted_status']['user']['listed_count'],tweet[i]['retweeted_status']['user']['favourites_count'],tweet[i]['retweeted_status']['user']['statuses_count'],tweet[i]['retweeted_status']['user']['created_at'],tweet[i]['retweeted_status']['user']['protected'] ))
                #print('Record inserted --------------------------------------------------------------------------------->')
                tweet_user_list.append(tweet[i]['retweeted_status']['user']['id_str'])
                inserted_rows_count = inserted_rows_count + 1
                      
        else:
            
            Skipped_rows_count = Skipped_rows_count + 1
            continue
            
    except Exception as e:
        print("Error encountered:",e)
        conn.commit()
        # Roll back the transaction
        conn.rollback()
        continue

conn.commit()

print("Inserted_rows_count: ",inserted_rows_count)
print("Skipped_rows_count: ",Skipped_rows_count)


Total rows of data available:  2001
Inserted_rows_count:  1851
Skipped_rows_count:  150


In [185]:
# Close the cursor and database connection
cursor.close()
conn.close()
